In [75]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import multiprocessing as mp
from multiprocessing import Pool, Value, Array
import time
from numba import njit
import numpy as np
import os
import multiprocessing
import random
import sys


In [61]:
def mc_pi(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [62]:
%%time

res = [mc_pi(int(1e6)) for i in range(2)]

KeyboardInterrupt: 

In [51]:
@njit()
def mc_pi_numba(n):
    s = 0
    for i in range(n):
        x = np.random.uniform(-1, 1)
        y = np.random.uniform(-1, 1)
        if (x**2 + y**2) < 1:
            s += 1
    return 4*s/n

In [52]:
%%time

res = [mc_pi_numba(int(1e6)) for i in range(2)]


CPU times: user 211 ms, sys: 7.98 ms, total: 219 ms
Wall time: 221 ms


In [53]:
np.array(res)

array([3.139124, 3.141576])

In [58]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(1000)])

KeyboardInterrupt: 

In [55]:
np.array(list(res))

array([3.140996, 3.142452, 3.14368 , 3.14292 , 3.141316, 3.139808,
       3.139864, 3.140724, 3.142452, 3.139852])

In [63]:
%%time

with ProcessPoolExecutor(max_workers=20) as pool:
    res = pool.map(mc_pi_numba, [int(1e4) for i in range(int(1e3))])

CPU times: user 1.26 s, sys: 140 ms, total: 1.4 s
Wall time: 2.59 s


In [57]:
%%time

with ProcessPoolExecutor(max_workers=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e4) for i in range(int(1e3))], chunksize=100)

CPU times: user 20.9 ms, sys: 24 ms, total: 45 ms
Wall time: 137 ms


In [66]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e7) for i in range(10)])

CPU times: user 26 ms, sys: 16 ms, total: 42 ms
Wall time: 754 ms


In [68]:
%%time

with mp.Pool(processes=4) as pool:
    res = pool.map(mc_pi_numba, [int(1e4) for i in range(int(1e4))])

CPU times: user 27.7 ms, sys: 40 ms, total: 67.8 ms
Wall time: 864 ms


In [69]:
def f(i):
    time.sleep(np.random.random())
    print(os.getpid(), i)

In [73]:
for i in range(10):
    p = mp.Process(target=f, args=(i,))
    p.start()
    p.join()

31177 0
31186 1
31197 2
31206 3
31215 4
31224 5
31233 6
31242 7
31252 8
31262 9


In [92]:
#
# Functions used by test code
#

def calculate(func, args):
    result = func(*args)
    return '%s says that %s%s = %s' % (
        multiprocessing.current_process().name,
        func.__name__, args, result
        )

def calculatestar(args):
    return calculate(*args)

def mul(a, b):
    time.sleep(0.5 * random.random())
    return a * b

def plus(a, b):
    time.sleep(0.5 * random.random())
    return a + b

def f(x):
    return 1.0 / (x - 5.0)

def pow3(x):
    return x ** 3

def noop(x):
    pass

def test():
    
    for j in range(6):
        PROCESSES = int(j+1) 
        print(PROCESSES)
        print('Creating pool with %d processes\n' % PROCESSES)

        test = np.random.rand(int(1e3),1)

        with multiprocessing.Pool(PROCESSES) as pool:

            #
            # Tests
            #

            TASKS = [(mul, (test[i], 7)) for i in range(len(test))] + \
                    [(plus, (test[i], 8)) for i in range(len(test))]

            results = [pool.apply_async(calculate, t) for t in TASKS]
            imap_it = pool.imap(calculatestar, TASKS)
            imap_unordered_it = pool.imap_unordered(calculatestar, TASKS)

            print('Ordered results using pool.apply_async():')
            for r in results:
                print('\t', r.get())
            print()

            print('Ordered results using pool.imap():')
            for x in imap_it:
                print('\t', x)
            print()

            print('Unordered results using pool.imap_unordered():')
            for x in imap_unordered_it:
                print('\t', x)
            print()

            print('Ordered results using pool.map() --- will block till complete:')
            for x in pool.map(calculatestar, TASKS):
                print('\t', x)
            print()

            #
            # Test error handling
            #

            print('Testing error handling:')

            try:
                print(pool.apply(f, (5,)))
            except ZeroDivisionError:
                print('\tGot ZeroDivisionError as expected from pool.apply()')
            else:
                raise AssertionError('expected ZeroDivisionError')

            try:
                print(pool.map(f, list(range(10))))
            except ZeroDivisionError:
                print('\tGot ZeroDivisionError as expected from pool.map()')
            else:
                raise AssertionError('expected ZeroDivisionError')

            try:
                print(list(pool.imap(f, list(range(10)))))
            except ZeroDivisionError:
                print('\tGot ZeroDivisionError as expected from list(pool.imap())')
            else:
                raise AssertionError('expected ZeroDivisionError')

            it = pool.imap(f, list(range(10)))
            for i in range(10):
                try:
                    x = next(it)
                except ZeroDivisionError:
                    if i == 5:
                        pass
                except StopIteration:
                    break
                else:
                    if i == 5:
                        raise AssertionError('expected ZeroDivisionError')

            assert i == 9
            print('\tGot ZeroDivisionError as expected from IMapIterator.next()')
            print()

            #
            # Testing timeouts
            #

            print('Testing ApplyResult.get() with timeout:', end=' ')
            res = pool.apply_async(calculate, TASKS[0])
            while 1:
                sys.stdout.flush()
                try:
                    sys.stdout.write('\n\t%s' % res.get(0.02))
                    break
                except multiprocessing.TimeoutError:
                    sys.stdout.write('.')
            print()
            print()

            print('Testing IMapIterator.next() with timeout:', end=' ')
            it = pool.imap(calculatestar, TASKS)
            while 1:
                sys.stdout.flush()
                try:
                    sys.stdout.write('\n\t%s' % it.next(0.02))
                except StopIteration:
                    break
                except multiprocessing.TimeoutError:
                    sys.stdout.write('.')
            print()
            print()


if __name__ == '__main__':
    multiprocessing.freeze_support()
    test()

1
Creating pool with 1 processes

Ordered results using pool.apply_async():
	 ForkPoolWorker-232 says that mul(array([0.86542343]), 7) = [6.05796403]
	 ForkPoolWorker-232 says that mul(array([0.37623195]), 7) = [2.63362363]
	 ForkPoolWorker-232 says that mul(array([0.60940408]), 7) = [4.26582859]
	 ForkPoolWorker-232 says that mul(array([0.40573679]), 7) = [2.8401575]
	 ForkPoolWorker-232 says that mul(array([0.34612612]), 7) = [2.42288282]
	 ForkPoolWorker-232 says that mul(array([0.65794188]), 7) = [4.60559313]
	 ForkPoolWorker-232 says that mul(array([0.43154359]), 7) = [3.02080514]
	 ForkPoolWorker-232 says that mul(array([0.8514159]), 7) = [5.95991133]
	 ForkPoolWorker-232 says that mul(array([0.96287585]), 7) = [6.74013097]
	 ForkPoolWorker-232 says that mul(array([0.42307667]), 7) = [2.96153669]
	 ForkPoolWorker-232 says that mul(array([0.01001895]), 7) = [0.07013262]
	 ForkPoolWorker-232 says that mul(array([0.26334234]), 7) = [1.84339638]
	 ForkPoolWorker-232 says that mul(arr

	 ForkPoolWorker-232 says that mul(array([0.48693032]), 7) = [3.40851227]
	 ForkPoolWorker-232 says that mul(array([0.04991767]), 7) = [0.34942369]
	 ForkPoolWorker-232 says that mul(array([0.13323124]), 7) = [0.93261865]
	 ForkPoolWorker-232 says that mul(array([0.37955293]), 7) = [2.65687052]
	 ForkPoolWorker-232 says that mul(array([0.63384112]), 7) = [4.43688781]
	 ForkPoolWorker-232 says that mul(array([0.56500035]), 7) = [3.95500247]
	 ForkPoolWorker-232 says that mul(array([0.64886867]), 7) = [4.54208067]
	 ForkPoolWorker-232 says that mul(array([0.40951658]), 7) = [2.86661609]
	 ForkPoolWorker-232 says that mul(array([0.28560451]), 7) = [1.99923159]
	 ForkPoolWorker-232 says that mul(array([0.07783459]), 7) = [0.54484216]
	 ForkPoolWorker-232 says that mul(array([0.91849659]), 7) = [6.4294761]
	 ForkPoolWorker-232 says that mul(array([0.36073648]), 7) = [2.52515537]
	 ForkPoolWorker-232 says that mul(array([0.26849483]), 7) = [1.8794638]
	 ForkPoolWorker-232 says that mul(array

	 ForkPoolWorker-232 says that mul(array([0.75391436]), 7) = [5.27740052]
	 ForkPoolWorker-232 says that mul(array([0.16886444]), 7) = [1.18205107]
	 ForkPoolWorker-232 says that mul(array([0.53987483]), 7) = [3.77912384]
	 ForkPoolWorker-232 says that mul(array([0.4978574]), 7) = [3.48500182]
	 ForkPoolWorker-232 says that mul(array([0.62021896]), 7) = [4.3415327]
	 ForkPoolWorker-232 says that mul(array([0.53613432]), 7) = [3.75294023]
	 ForkPoolWorker-232 says that mul(array([0.32462188]), 7) = [2.27235319]
	 ForkPoolWorker-232 says that mul(array([0.79190006]), 7) = [5.54330039]
	 ForkPoolWorker-232 says that mul(array([0.17973184]), 7) = [1.25812286]
	 ForkPoolWorker-232 says that mul(array([0.38152861]), 7) = [2.67070029]
	 ForkPoolWorker-232 says that mul(array([0.0991536]), 7) = [0.69407522]
	 ForkPoolWorker-232 says that mul(array([0.1409131]), 7) = [0.98639169]
	 ForkPoolWorker-232 says that mul(array([0.88128266]), 7) = [6.16897864]
	 ForkPoolWorker-232 says that mul(array([

	 ForkPoolWorker-232 says that mul(array([0.53376136]), 7) = [3.73632953]
	 ForkPoolWorker-232 says that mul(array([0.83151223]), 7) = [5.8205856]
	 ForkPoolWorker-232 says that mul(array([0.3846256]), 7) = [2.6923792]
	 ForkPoolWorker-232 says that mul(array([0.87819718]), 7) = [6.14738027]
	 ForkPoolWorker-232 says that mul(array([0.74436409]), 7) = [5.21054862]
	 ForkPoolWorker-232 says that mul(array([0.69255428]), 7) = [4.84787999]
	 ForkPoolWorker-232 says that mul(array([0.09799824]), 7) = [0.68598768]
	 ForkPoolWorker-232 says that mul(array([0.88668078]), 7) = [6.20676544]
	 ForkPoolWorker-232 says that mul(array([0.13552504]), 7) = [0.94867527]
	 ForkPoolWorker-232 says that mul(array([0.07159399]), 7) = [0.50115795]
	 ForkPoolWorker-232 says that mul(array([0.8624342]), 7) = [6.0370394]
	 ForkPoolWorker-232 says that mul(array([0.03458035]), 7) = [0.24206248]
	 ForkPoolWorker-232 says that mul(array([0.50276325]), 7) = [3.51934274]
	 ForkPoolWorker-232 says that mul(array([0

Process ForkPoolWorker-232:
Traceback (most recent call last):
  File "/home/thiago/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/thiago/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/thiago/anaconda3/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "<ipython-input-92-ea54b0c1f1b6>", line 6, in calculate
    result = func(*args)
  File "<ipython-input-92-ea54b0c1f1b6>", line 16, in mul
    time.sleep(0.5 * random.random())
KeyboardInterrupt


	 ForkPoolWorker-232 says that mul(array([0.59622445]), 7) = [4.17357114]


KeyboardInterrupt: 

In [85]:
from numpy import random

print (vals)

IndexError: index 1 is out of bounds for axis 0 with size 1